In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h2> Table of Content </h2>
<ul> <li> <a href="#preperation"> Preparation </a> </li>
    <li> <a href="#importdata"> Load the datas </a> </li>
    <li> <a href="#look"> First Look at the data</a> </li>
    <li> <a href="#exploration"> Datas explorations</a></li>
    <li> <a href="#Modeling"> Modeling </a> </li>
    </ul>
<hr>

<h2 id="preparation">Preparation</h2>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt 
import os
import plotly_express as px
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
import matplotlib.patches as mpatches
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import TweetTokenizer , word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , GRU,Input,TimeDistributed , Embedding,Bidirectional,\
BatchNormalization,Dropout,LSTM ,GlobalAveragePooling1D
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy ,categorical_crossentropy
from nltk.stem import PorterStemmer
from collections import Counter
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import cross_val_score ,KFold
import numpy as np 
from sklearn.metrics import roc_auc_score, accuracy_score ,auc , roc_curve

<h2 id=importdata>Load the datas</h2>

In [ ]:
root="/kaggle/input/nlp-getting-started/"
train_df=pd.read_csv(os.path.join(root,"train.csv"))
test_df=pd.read_csv(os.path.join(root,"test.csv"))
df_sub=pd.read_csv(os.path.join(root,"sample_submission.csv"))

<h2 id="#look">First Look at the data

In [ ]:
# Take a look at the datas 
train_df.head()

In [ ]:
# info about training datas
train_df.info()

In [ ]:
# info about test datas
test_df.info()

<h2 id="

==> It obviously, that we should handle the missing values of the variables location and keyword.

In [ ]:
#Some statistics on the datas.
train_df.describe()

<h2 id="#exploration"> Datas explorations </h2>

<h3> Let's Ckeck Target Didtribution</h3>

#### Check For Datas Imbalance

In [ ]:
fig=px.pie(train_df,names="target",title="Target Disributions")
fig.update_layout(title={"x":0.475,"y":0.9,"xanchor":"center","yanchor":"top"})
fig.show()

<h3> Tweets Location </h3>

In [ ]:
nb=len(train_df['location'].unique())
print("Tweets come from {} differents locations ".format(nb))

<h3> Top 20 Locations </h3>

In [ ]:
top_20=pd.DataFrame(train_df['location'].value_counts()[:20]).reset_index(drop=False)
top_20.rename(columns={'index':"Locations","location":"Occurences"},inplace=True)

In [ ]:
fig=px.bar(top_20,x="Locations",y="Occurences",orientation='v',color="Occurences",title=\
          "Top 20 most used Countries ")
fig.update_layout(title={"x":0.475,"y":0.9,"xanchor":"center","yanchor":"top"})
fig.show()

In [ ]:
df=train_df[train_df['location'].notna()]

In [ ]:
lb=LabelEncoder()
df['location']=lb.fit_transform(df['location'])

In [ ]:
X=df['location']
Y=df['target']

#T_sne implementation

X_reduced=TSNE(n_components=2,random_state=42).fit_transform(X.values.reshape(-1,1))

In [ ]:
f,ax=plt.subplots(1,1,figsize=(12,8))
blue_patch = mpatches.Patch(color='#0A0AFF' ,label='Not Fake')
red_patch = mpatches.Patch(color='#AF0000' ,label='Fake')

plt.scatter(X_reduced[:,0],X_reduced[:,1],c=(Y==1),label=["Not Fake"],cmap="coolwarm"\
            )
plt.scatter(X_reduced[:,0],X_reduced[:,1],c=(Y==0),label=["Fake"],cmap="coolwarm"\
           )
plt.grid(True)
plt.legend(handles=[blue_patch,red_patch])
f.suptitle('Clusters using Dimmensionality Reduction',fontsize=14)
plt.title("t-SNE")

<h2 id=

<font color=redblue> <b>the above t-SNE clustering shows that fake and real tweets can come from any locations. So the location can not be an important features to classify the tweets. </font>

<h3> KeyWords </h3>

#### Top 20 KeyWords

In [ ]:
# most used word for disaster tweet
df=train_df[train_df['target']==1]
top_20=pd.DataFrame(df['keyword'].value_counts()[:20]).reset_index(drop=False)
top_20.columns=['keywords','occurences']

In [ ]:
fig=px.bar(top_20,x="keywords",y="occurences",color='occurences',title=\
          "Top 20 most used keyword for disaster tweets")
fig.update_layout(title={"x":0.475,"y":0.9,"xanchor":"center","yanchor":\
                        "top"})
fig.show()

In [ ]:
df=train_df[train_df["keyword"].notna()]

In [ ]:
le=LabelEncoder()
df['keyword']=le.fit_transform(df['keyword'])

In [ ]:
X=df['keyword']
Y=df['target']
#T-SNE implementation
X_reduced=TSNE(n_components=2,random_state=41).fit_transform(\
                                                             X.values.reshape(-1,1))

In [ ]:
f,ax=plt.subplots(1,1,figsize=(12,8))
blue_patch=mpatches.Patch(color='#0A0AFF',label='Not Fake')
red_patch=mpatches.Patch(color='#AF0000',label='Fake')

f.suptitle("Clusters using Dimensionnality Reduction ")

ax.scatter(X_reduced[:,0],X_reduced[:,1],c=(Y==1),label="Not Fake",\
          cmap="coolwarm")
ax.scatter(X_reduced[:,0],X_reduced[:,1],c=(Y==0),label="Kake",\
          cmap="coolwarm")
ax.grid(True)
ax.legend(handles=[blue_patch,red_patch])
plt.title("T-SNE")

<font color=redblue> The T-SNE clustering above, shows a distingushed clusters for real disaster tweets and fake disaster tweets, which confirm that there is distingushed keywords for each of these cluster.

<h3> text analysis </h3>

In [ ]:
tx=" ".join(df['text'].values.tolist())

In [ ]:
fig,ax1=plt.subplots(1,1,figsize=(12,16))
wordcl=WordCloud().generate(tx)
plt.imshow(wordcl,interpolation="bilinear")
plt.axis("off")
plt.title("Most used words for real disaster tweets",fontsize=16)
plt.show()

<h3> Text Modeling Topic </h3>

In [ ]:
tx=train_df['text'].values.tolist()

In [ ]:
count_vectoriser=CountVectorizer(stop_words="english",max_df=0.95,min_df=2)
tf=count_vectoriser.fit_transform(tx)

In [ ]:
#GridSearch to choice the number of topics to extract
params={'n_components':[5,10,20,50]}
lda=LatentDirichletAllocation()
model=GridSearchCV(lda,params)
model.fit(tf)

In [ ]:
print("Best Model's Params :{}".format(model.best_params_))

In [ ]:
model=model.best_estimator_

In [ ]:
nb_topics=5
def display_topic(md,nb_topics,features,no_words):
    for topic_idx ,topic in enumerate(md.components_):
        print("Topic {} :".format(topic_idx))
        print(" ".join(features[j] for j in topic.argsort()[-no_words-1:-1]))
        

In [ ]:
# Display the most 5 represented topic in our corpus
features=count_vectoriser.get_feature_names()
no_words=10
display_topic(model,nb_topics,features,no_words)

<h2 id="Modeling"> Modeling </h2>

<h3> Naives Bayes: Baseline</h3>

In [ ]:
def tokenizer(x):
    token = TweetTokenizer().tokenize(x)
    return token
def stem(x):
    stemmer = PorterStemmer()
    res = [stemmer.stem(elt) for elt in x]
    return res 

In [ ]:
# Compute the frequency apparition of each word in our corpus
stop_words = Counter([x for row in train_df['text'].values for x in row.split(" ")])

In [ ]:
# We consider the 100 most used words as common words.
common_words = list(zip(*stop_words.most_common(100)))[0]

In [ ]:
common_words = set(common_words)

In [ ]:
common_words.update(nltk.corpus.stopwords.words("english"))

In [ ]:
training=train_df['text'].values.tolist()
test=test_df['text'].values.tolist()

In [ ]:
countvectorizer=TfidfVectorizer(tokenizer=tokenizer,\
                               max_df=0.95,min_df=2)

In [ ]:
X_tr=countvectorizer.fit_transform(training)
X_pr=countvectorizer.fit_transform(test)

In [ ]:
Y_tr=train_df["target"].values

In [ ]:
xtr,xts,ytr,yts=train_test_split(X_tr,Y_tr,test_size=0.2,stratify=Y_tr)

In [ ]:
#class_weights=class_weight.compute_class_weight("balanced",\
                                                #np.unique(Y_tr),Y_tr)

In [ ]:
estimator=MultinomialNB()
estimator.fit(xtr,ytr)

In [ ]:
# estimate performance accuracy
sc=estimator.score(xts,yts)
print("The score accuracy of the Naive bayes estimator is : {}".\
      format(sc))

<h3> Naives Bayes: fine tunnned

In [ ]:
def preprocessing(x):
    """
    - Lowercase the sentence. 
    - Change 't to 'not'.
    - Remove @name
    - Remove all stop words unless 'can' and 'not'.
    - Remove all punctuation and special charecter except '!'
    - Remove trailing white space.
    """
    x = x.lower()
    x = re.sub(r"(@.?)[\s]"," ",x)
    x = re.sub(r"\'t","not",x)
    x= " ".join([word for word in x.split(" ") if word not in common_words or word in ("can","not")])
    x= re.sub(r"([\?\\\,\:\;\.\'\"\(\)])"," ",x)
    x= re.sub(r"\s+"," ",x).strip()
    return x
    

In [ ]:
# Apply our preprocessing treatment to our training datas.
training = [preprocessing(text) for text in train_df['text'].values]

In [ ]:
# Define TF_IDF trasnformer with the new subtility to take en consideration ngram from 1 to 3.
tf_idf = TfidfVectorizer(max_df=0.95,min_df=2)

In [ ]:
X = tf_idf.fit_transform(train_df['text'].values)

In [ ]:
xtr,xts,ytr,yts = train_test_split(X,Y_tr,test_size=0.2,stratify=Y_tr)

In [ ]:
def get_auc_cv(model):
    kf = KFold(5,shuffle=True,)
    auc = cross_val_score(model,xtr,ytr,cv=kf,scoring="roc_auc")
    
    return auc.mean()

In [ ]:
res = pd.Series([get_auc_cv(MultinomialNB(i)) for i in np.arange(1,10,0.1)],index=np.arange(1,10,0.1))

In [ ]:
best_alpha = res.idxmax()
print("Best_alpha : {}".format(best_alpha))

plt.plot(res)
plt.title("AUC vs.Alpha")
plt.xlabel("alpha")
plt.ylabel("AUC")
plt.show()

In [ ]:
def evaluate_roc(probs,y_true):
    """ 
    - Print AUC and accuracy score
    - plot Roc curve.
    @params: probs(np.array):an array of predicted probabilities with shape (len(y_true,2)).
    @params: y_true : an array of the true value with shape (len(y_true),)
    """
    prob = probs[:,1]
    fpr,tpr,threshold = roc_curve(y_true,prob)
    AUC = auc(fpr,tpr)
    print("AUC:{}".format(AUC))
    
    # Get the accuracy score.
    ypred = np.where(prob>0.5,1,0)
    accuracy = accuracy_score(y_true,ypred)
    print("The accuracy score: {}".format(accuracy))
    
    # Plot the roc curve 
    plt.plot(fpr,tpr,'b',label='AUC')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.legend(loc="best")
    plt.title("Receiver Operating Characteristic")
    plt.show()

In [ ]:
model = MultinomialNB(alpha=best_alpha) # define Naives bayes with the best hyperparameter.
model.fit(xtr,ytr)
probs = model.predict_proba(xts)

In [ ]:
evaluate_roc(probs,yts)

<font color=redblue> The Gread search to select the best hyperparameter for our model , had enhanced slightly the performance of our model. We should mention that neither the processing of datas as detailed in the function named preprocessing, nor the the n_gram with range 1 to 3, had positive effect on the performance. </font> 

<h3>RNN Model </h3>

In [ ]:
# Mandatory functions to prepare datas
voc_size = 20000
max_len=100
def tokenize(x):
    tk=Tokenizer(char_level=False)
    tk.fit_on_texts(x)
    return tk.texts_to_sequences(x),tk
def pad(x,max_len=None):
    if max_len is None:
        l=[len(s) for s in x]
        max_len=max(l)
    return pad_sequences(x,maxlen=max_len,padding='pre')
       
def preprocessing(x):
    seq_x,tk=tokenize(x)
    return  pad(seq_x),tk

In [ ]:
# Define the model
def model(input_shape,size,emb_size):
    lr=1e-3
    md= Sequential()
    md.add(Embedding(size,emb_size,input_length=input_shape[1]))
    md.add(BatchNormalization())
    md.add(Bidirectional(GRU(50,return_sequences=True),\
                         input_shape = (emb_size,1)))
    
    md.add(BatchNormalization())
    md.add(Dropout(0.3))
    md.add(TimeDistributed(Dense(50,activation="softmax")))
    md.add(BatchNormalization())
    md.add(Dropout(0.1))
    md.add(Dense(1,activation="sigmoid"))
    md.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(lr),metrics=["accuracy"])
    return md

In [ ]:
#Prepare datas
inp=train_df['text'].tolist()
X,tk=preprocessing(inp)

In [ ]:
#Reshape datas
X=X.reshape(-1,X.shape[1],1)
input_shape=X.shape

In [ ]:
#Corpus size
size=len(tk.word_index)

In [ ]:
#Build Model
model= model(input_shape,size+1,50)

In [ ]:
Y=train_df["target"].values

In [ ]:
model.fit(X,Y,batch_size=32,shuffle=True,validation_split=0.1,epochs=10)

<font color=redblue> The RNN model don't give better result than naivebayes. Indeed, he barely gives result near to the result of the random estimator. Therefore, The RNN model don't adapted to such binair classification. </font>

<h3> Fine-Tunning Bert

<h4> Import Required Librairies:

In [ ]:
!pip install GPUtil

In [ ]:
from transformers import BertForSequenceClassification,BertTokenizer,AdamW,\
get_linear_schedule_with_warmup,BertModel
import torch 
from torch.utils.data import DataLoader,TensorDataset,SequentialSampler
from GPUtil import showUtilization as gpu_usage
from numba import cuda


In [ ]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

<h4>Tokenization and Input Formatting:

> Hereunder, we will perform some slight preprocessing on our text removing entity mentions and special character. The level of processing here is much less than in previous approach because BERT was trained with the entire sentence.

In [ ]:
def preprocessing(text):
    """ 
    - Remove entity mention @
    - Remove some special characters (exple: #)
    - Replace 't by not.
    
    @param text(str) : an introduced text to be processed.
    @param text(str) : a processed text.
    """
    # Remove '@'
    text = re.sub(r"(@.*?)[\s]"," ",text)
    # Remove '#'
    text = re.sub(r"(#.*?)[\s]"," ",text)
    # Remove trailing whitespace
    text = re.sub(r"\s+"," ",text).strip()
    
    return text

In order to apply the pre-trained BERT, we must use the tokenizer provided by the library. This is because (1) the model has a specific, fixed vocabulary and (2) the BERT tokenizer has a particular way of handling out-of-vocabulary words.

In addition, we are required to add special tokens to the start and end of each sentence, pad & truncate all sentences to a single constant length, and explicitly specify what are padding tokens with the "attention mask".

The encode_plus method of BERT tokenizer will:

(1) split our text into tokens,

(2) add the special [CLS] and [SEP] tokens, and

(3) convert these tokens into indexes of the tokenizer vocabulary,

(4) pad or truncate sentences to max length, and

(5) create attention mask.

In [ ]:
# Load the Bert Tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)

In [ ]:
def preprocessing_for_bert(data,Max_len):
    """Perform required preprocessing step for pretrained BERT.
    @param data(np.array) : Array of texts to be processed.
    
    @ return inp_ids(torch.Tensor): Tensor of token ids to be fed to the model.
    @ return mask_attention(torch.Tensor): Tensor of indices specifying which tokens 
                                           should be attended by the model
    
    """
    
    # Create empty lists to store outputs.
    
    inp_ids = []
    mask_attention = []
    
    for sent in data :
        
        encoded_sent = tokenizer.encode_plus(text=preprocessing(sent),
                                            add_special_tokens=True,
                                            padding='max_length',
                                            truncation=True,
                                            max_length=Max_len)
        inp_ids.append(encoded_sent.get('input_ids'))
        mask_attention.append(encoded_sent.get('attention_mask'))
    
    # convert list to tensors
    inp_ids = torch.tensor(inp_ids)
    mask_attention = torch.tensor(mask_attention)  
    
    return inp_ids,mask_attention

Before tokenizing,we need to specify the maximum length of our sentences

In [ ]:
#Concatenate train and test data.
data = np.concatenate([train_df["text"],test_df["text"]],axis=0)
#Encode our concatenated data.
encoded_tweets = [tokenizer.encode(sent,add_special_tokens=True) for sent in data]
# Find the maximum length.
Max_len = max([len(sent) for sent in encoded_tweets])
print('Max_length: ',Max_len)

<h4> Set up GPU for training :

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are {} GPU(s) available".format(torch.cuda.device_count()))
    print("Device name:",torch.cuda.get_device_name(0))
else:
    print("No GPU available,using CPU instead")
    device = torch.device("cpu")

<h4> Create BertClassifier

In [ ]:
import torch.nn as nn 

class BertClassifier(nn.Module):
    """ Bert Model for classification Tasks."""
    def __init__(self,freeze_bert=False):
        super(BertClassifier,self).__init__()
    
        self.bert = BertModel.from_pretrained("bert-base-uncased")
    
        #Specify hidden size of bert, hidden size of our classifier,and numbers of labels.
        D_in, H ,D_out = 768,50,2
    
        self.classifier = nn.Sequential(nn.Linear(D_in,H),
                                   nn.ReLU(),
                                   nn.Dropout(0.3),
                                   nn.Linear(H,D_out))
    
        if freeze_bert :
           for param in self.bert.parameters():
              param.requires_grad = False 
    def forward(self,input_id,attention_mask):
        """ Feed input to Bert and the classifier to compute output.
            
            @param input_id(torch tensor): an input tensor with shape (batch_size,Max_len)
            @param attention_mask (torch tensor) : an input tensor with shape (batch_size,Max_len)
            @param output (torch tensor): an output tensor with shape (batch_size,num_labels)
        
        """
        
        hidden_states=self.bert(input_id,attention_mask)
        hidden_states = hidden_states[0][:,0,:]
        
        output = self.classifier(hidden_states)
        
        return output
    
    

<h4> Optimizer & Learning rate Scheduler:

To fine-tunne our Bert Classifier, we need to create an optimizer.The authors recommend following hyper-parametrs:

   <ul>
    <li> batch_size : 16 or 32 </li>
    <li> optimizer : Adam </li>
    <li> Learning rate : 5e-5,3e-5,2e-5
    <li> Number of epochs : 2,3
   </ul>

In [ ]:
def initialize_model(traindataloader,epochs=4):
    """ Initialize the Bert Model, the optimizer and the learning rate scheduler."""
    # Instantiate Bert classifier.
    bertmodel = BertClassifier(freeze_bert=False)
    if torch.cuda.device_count() > 1:
        bertmodel = nn.DataParallel(bertmodel)
    bertmodel.to(device)
    #Create the optimizer
    optimizer = AdamW(bertmodel.parameters(),
                     lr=5e-5, # default value
                     eps=1e-8) # default value
    # Total number of training steps
    total_steps = len(traindataloader)*epochs
    
    # Set up the learning rate Scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_training_steps=total_steps,
                                               num_warmup_steps=0)
    
    return bertmodel,optimizer,scheduler

<h4> Create Pytorch DataLoader

In [ ]:
x_tr,x_ts,y_tr,y_ts = train_test_split(training,Y_tr,test_size=0.2,stratify=Y_tr)

In [ ]:
# compute inputs_ids and mask_attention tensors for training datas.
tr_ids,tr_mask = preprocessing_for_bert(x_tr,Max_len)
# compute inputs_ids and mask_attention tensors for test datas.
val_ids,val_mask = preprocessing_for_bert(x_ts,Max_len)
# Convert train label vector to torch tensor.
tr_labels = torch.tensor(y_tr)
val_labels = torch.tensor(y_ts)

# For fine tunning bert , the authors recommend a batch size of 16 or 32.
batch_size =32

#Create the Dataloader for our training set.
train_data = TensorDataset(tr_ids,tr_mask,tr_labels)
tr_sampler = SequentialSampler(train_data)
tr_dataloader = DataLoader(train_data,sampler=tr_sampler,batch_size=batch_size)

#Create the Dataloader for our validation set.
val_data = TensorDataset(val_ids,val_mask,val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data,sampler=val_sampler,batch_size=batch_size)

<h4> Training Loop:

In [ ]:
import time 
#define Loss function
loss_fn= nn.CrossEntropyLoss()

def train(model,optim,scheduler,tr_data,val_data,evaluation=True,epochs=4):
    """ Training the Bert Classifier"""
    
    #Start training loop.
    print("Start training...\n")
    for epoch_i in range(epochs):
        #==============================================
        #                 Training
        #==============================================
        
        model.train()
        
        # Mesure the elapsed time of each epoch.
        t0_epoch,t0_batch = time.time(),time.time()
        # Iinitialize valiation tracked values.
        val_loss,val_accuracy = None,None
        # Reset tracking variables at the beginning of each epoch.
        loss_batch,loss_total, batch_counts= 0,0,0
        for i,data in enumerate(tr_data):
            batch_counts +=1
            #Load input datas to device
            inp,mask,lab = tuple(t.to(device) for t in data)
            
            #Zero out any previously calculated gradients.
            model.zero_grad()
            #Perform a forward pass.This will return logits.
            output = model(inp,mask)
            # Compute loss and accumulate the loss values. 
            loss = loss_fn(output,lab)
            loss_batch += loss.item()
            loss_total += loss.item()
            
            #Perform a backward pass to calculate gradients.
            loss.backward()
            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # update weights and the learning rate 
            optim.step() 
            scheduler.step()
            optim.zero_grad()
            
            # print the loss values and the elapsed time for each 20 steps.
                                           
            if (i % 20 ==0 and i!=0) or (i == len(tr_data)-1):
                print("Epochs : {} | Batch :{} | Batch_loss :{} | Total_loss: - | Val_loss : - | Val_Accuracy: - | elapsed_time : {}".format(epoch_i,(i+1)//20,\
                                                                         loss_batch/batch_counts,time.time()-t0_batch))
                
                #Reset tracking variables values on the end of each batch.
                t0_batch = time.time()
                batch_counts = 0
                loss_batch = 0
            del(inp)
            del(mask)
            del(lab)
            del(output)
        print("-"*120)
        if evaluation :
           val_loss,val_accuracy = validation(model,val_data)
        # print the loss values and the elapsed time for each epoch.
        print("Epochs : {} | Batch : - | Batch_loss : - | Toal_loss : {} | Val_loss : {} | Val_Accuracy : {} | elapsed_time : {}".format(epoch_i,loss_total/len(tr_data),val_loss,val_accuracy,time.time()-\
                                                 t0_epoch))
        print("-"*120)
        # Reset tracking variables values at the end of each epoch.
        t0_epoch = 0
        loss_total = 0
    print("Training complete!")
def validation(model,val_data):
    """ Mesure the model'performances on our validation set."""
    
    # Put the model into the evaluation mode.The dropout layers are disabled during
    # validation test
    model.eval()
    val_loss = 0
    val_accuracy = 0
    for data in val_data : 
        # Load data to device                                  
        inp , msk ,labels= (t.to(device) for t in data)
        # Compute logits
        with torch.no_grad():
           out = model(inp,msk)
        # Compute the loss values.                                   
        loss = loss_fn(out,labels)
        val_loss += loss.item()
        
        # Get the predictions.
        _ ,predictions = torch.max(out,dim=1)
                                           
        # Calculate the accuracy score.
        accuracy = torch.sum(predictions==labels)
        val_accuracy += accuracy.item()
       
                                           
    val_loss = val_loss / len(val_data)
    accuracy = val_accuracy / len(y_ts)                                 
    
    return val_loss,accuracy
        
                                          
            
        

In [ ]:
free_gpu_cache()

In [ ]:
# Now , let's training our model.
Bertclassifier,optimizer,scheduler = initialize_model(tr_dataloader,epochs=2)

In [ ]:
train(Bertclassifier,optimizer,scheduler,tr_dataloader,val_dataloader,evaluation=True,epochs=2)

<h4> Validation on Validation Set :

In [ ]:
from torch.nn.functional import softmax
def bert_predict_proba(model,val_dataloader):
    """ Perform a forward pass on the introduced model to predict probabilities"""
    # Put the model into the evaluation mode. The dropout layers are disabled during 
    # the test time.
    model.eval()
    
    all_predictions = []
    
    for datas in val_dataloader:
        # Load the datas to device.
        inp_id,msk_att,lab = tuple(t.to(device) for t in datas)
        
        # Compute logits
        with torch.no_grad():
            output = model(inp_id,msk_att)
        
        all_predictions.append(output)
        
    # Concatenate logits from each batch.
    all_logits = torch.cat(all_predictions,dim=0)
    
    # Apply softmax to calculate probabilities
    probs = softmax(all_logits,dim=1).cpu().numpy()
    
    
    return probs   
    
        
            

In [ ]:
# Compute the predicted probabilities on the test set 
probabilities = bert_predict_proba(Bertclassifier,val_dataloader)

In [ ]:
# Evaluate the Bertclassifier
evaluate_roc(probabilities,y_ts)

<font color=red> <b> ! PLease upvote to carry on , if you like it. </font>